[example-logistic-regression](https://github.com/torch/demos/blob/master/logistic-regression/example-logistic-regression.lua)


In [26]:
require 'nn'
require 'optim'

In [27]:
local matio = require 'matio'
data = matio.load('ex3data1.mat')

In [28]:
trainset = {}

In [29]:
trainset.data = data.X

In [30]:
mean = {}
stdv = {}
for i=1,400 do
    mean[i] = trainset.data[{ {},{i} }]:mean()
    stdv[i] = trainset.data[{ {}, {i} }]:std()
    --print(i .. 'th mean: ' .. mean[i])
    --print(i .. 'th std dev: ' .. stdv[i])
    trainset.data[{ {},{i} }]:add(-mean[i])
    if stdv[i] ~= 0 then
        trainset.data[{ {},{i} }]:div(stdv[i])
    end
end

In [56]:
trainset.label = torch.zeros(5000,10)

In [57]:
for i=1,5000 do
    local idx = data.y[i]
    trainset.label[{ {i},{idx} }] = 1
end

In [58]:
numOutput = torch.max(trainset.label) - torch.min(trainset.label) + 1

In [64]:
-- ignore setmetatable for now, it is a feature beyond the scope of this tutorial. It sets the index operator.
setmetatable(trainset,
    {__index = function(t,i)
                return {t.data[i], t.label[i]}
        end}
);

function trainset:size()
    return self.data:size(1)
end

In [59]:
model = nn.Sequential()
model:add(nn.Linear(400,10))
model:add(nn.Sigmoid())
model:add(nn.LogSoftMax())

In [60]:
criterion = nn.ClassNLLCriterion()

In [61]:
trainer = nn.StochasticGradient(model, criterion)
trainer.learningRate = 0.001
trainer.maxIteration = 1e2

In [62]:
trainset

{
  data : DoubleTensor - size: 5000x400
  label : DoubleTensor - size: 5000x10
}


In [ ]:
trainer:train(trainset)

# StochasticGradient: training	


# current error = 2.1906496572094	


# current error = 2.0170610220783	


# current error = 1.9229929787112	


# current error = 1.8661224415631	


...

# current error = 1.5088889290138	


# current error = 1.5079375752326	


# current error = 1.5070228651073	


# current error = 1.5061421057322	


# current error = 1.5052930759053	


In [13]:
acc = 0.0
for i = 1, (#dataset_inputs)[1] do
    local groundtruth = trainset.label[i]
    local prediction = model:forward(trainset.data[i])
    local confidences, indices = torch.sort(prediction, true) -- true means descending order
    if groundtruth == indices[1] then
        acc = acc + 1
    end
end
print('accuracy: ' .. 100*acc/(#dataset_inputs)[1] .. '%')

[string "acc = 0.0..."]:2: attempt to get length of global 'dataset_inputs' (a nil value)
stack traceback:
	[string "acc = 0.0..."]:2: in main chunk
	[C]: in function 'xpcall'
	/root/torch/install/share/lua/5.1/itorch/main.lua:179: in function </root/torch/install/share/lua/5.1/itorch/main.lua:143>
	/root/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/root/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/root/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	[string "arg={'/root/.local/share/jupyter/runtime/kern..."]:1: in main chunk: 